<a href="https://colab.research.google.com/github/preetamjumech/Optuna-Hyperparameter-tuning/blob/main/Hyperparameter_tuning_using_Optuna__Bayesian_search_14_11_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.4 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')

Training set shape: (537, 8)
Test set shape: (231, 8)


In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize

In [18]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2024-11-14 14:53:03,193] A new study created in memory with name: no-name-fc1a5a59-2ac5-43ab-859c-20d8ad0f33ac
[I 2024-11-14 14:53:04,183] Trial 0 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 103, 'max_depth': 20}. Best is trial 0 with value: 0.7728119180633147.
[I 2024-11-14 14:53:05,189] Trial 1 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 184, 'max_depth': 13}. Best is trial 0 with value: 0.7728119180633147.
[I 2024-11-14 14:53:05,712] Trial 2 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 102, 'max_depth': 4}. Best is trial 0 with value: 0.7728119180633147.
[I 2024-11-14 14:53:06,110] Trial 3 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 72, 'max_depth': 7}. Best is trial 0 with value: 0.7728119180633147.
[I 2024-11-14 14:53:06,572] Trial 4 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 80, 'max_depth': 8}. Best is trial 0 with value: 0.772811918

In [19]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7839851024208566
Best hyperparameters: {'n_estimators': 119, 'max_depth': 15}


In [20]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


#Random Search using optuna

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize

In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2024-11-14 14:48:12,415] A new study created in memory with name: no-name-4890884c-d71a-494a-8901-3457efa1dc72
[I 2024-11-14 14:48:12,967] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 91, 'max_depth': 7}. Best is trial 0 with value: 0.7690875232774674.
[I 2024-11-14 14:48:13,492] Trial 1 finished with value: 0.7560521415270017 and parameters: {'n_estimators': 107, 'max_depth': 3}. Best is trial 0 with value: 0.7690875232774674.
[I 2024-11-14 14:48:14,153] Trial 2 finished with value: 0.7579143389199254 and parameters: {'n_estimators': 135, 'max_depth': 3}. Best is trial 0 with value: 0.7690875232774674.
[I 2024-11-14 14:48:14,882] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 135, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2024-11-14 14:48:15,610] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 132, 'max_depth': 11}. Best is trial 0 with value: 0.76908752

In [11]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 132, 'max_depth': 7}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.75


#Grid Search using optuna

In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [14]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2024-11-14 14:50:49,963] A new study created in memory with name: no-name-8e5cd9f3-1928-443f-bde6-7c5577d3c2d0
[I 2024-11-14 14:50:51,062] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2024-11-14 14:50:52,399] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2024-11-14 14:50:52,694] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2024-11-14 14:50:53,248] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2024-11-14 14:50:53,804] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [15]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.74


#Optuna visualization on bayesian search ( run those above cells again)



In [21]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [22]:
# 1. Optimization History
plot_optimization_history(study).show()

In [23]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [24]:
# 3. Slice Plot
plot_slice(study).show()

In [25]:
# 4. Contour Plot
plot_contour(study).show()

In [26]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

# Optimizing multiple ML models

In [27]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [28]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [29]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-11-14 15:12:30,250] A new study created in memory with name: no-name-8ae29b27-d00e-4cad-8ad6-3a2780a65b71
[I 2024-11-14 15:12:31,798] Trial 0 finished with value: 0.7616387337057727 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 77, 'learning_rate': 0.10377052160092574, 'max_depth': 8, 'min_samples_split': 8, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.7616387337057727.
[I 2024-11-14 15:12:34,124] Trial 1 finished with value: 0.7616387337057727 and parameters: {'classifier': 'RandomForest', 'n_estimators': 289, 'max_depth': 20, 'min_samples_split': 4, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 0 with value: 0.7616387337057727.
[I 2024-11-14 15:12:34,746] Trial 2 finished with value: 0.7579143389199254 and parameters: {'classifier': 'RandomForest', 'n_estimators': 84, 'max_depth': 13, 'min_samples_split': 9, 'min_samples_leaf': 3, 'bootstrap': True}. Best is trial 0 with value: 0.7616387337057727.
[I 2024-11-14 15:12:34,793] Trial 3 f

In [30]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.13589080717397212, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [31]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.761639,2024-11-14 15:12:30.258976,2024-11-14 15:12:31.798229,0 days 00:00:01.539253,NaN,NaN,GradientBoosting,NaN,NaN,0.103771,8.0,6.0,8.0,77.0,COMPLETE
1,1,0.761639,2024-11-14 15:12:31.806115,2024-11-14 15:12:34.123631,0 days 00:00:02.317516,NaN,False,RandomForest,NaN,NaN,NaN,20.0,1.0,4.0,289.0,COMPLETE
2,2,0.757914,2024-11-14 15:12:34.131567,2024-11-14 15:12:34.745885,0 days 00:00:00.614318,NaN,True,RandomForest,NaN,NaN,NaN,13.0,3.0,9.0,84.0,COMPLETE
3,3,0.785847,2024-11-14 15:12:34.747675,2024-11-14 15:12:34.793040,0 days 00:00:00.045365,2.190172,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
4,4,0.761639,2024-11-14 15:12:34.795104,2024-11-14 15:12:35.328831,0 days 00:00:00.533727,NaN,True,RandomForest,NaN,NaN,NaN,17.0,4.0,3.0,101.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.776536,2024-11-14 15:13:08.397069,2024-11-14 15:13:08.472993,0 days 00:00:00.075924,0.194204,NaN,SVM,auto,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.785847,2024-11-14 15:13:08.475993,2024-11-14 15:13:08.534093,0 days 00:00:00.058100,0.273067,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.789572,2024-11-14 15:13:08.540952,2024-11-14 15:13:08.595518,0 days 00:00:00.054566,0.125618,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.735568,2024-11-14 15:13:08.603985,2024-11-14 15:13:08.676937,0 days 00:00:00.072952,0.694213,NaN,SVM,auto,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [32]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,78
RandomForest,12
GradientBoosting,10


In [33]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.748603
RandomForest,0.765673
SVM,0.776274


In [34]:
# 1. Optimization History
plot_optimization_history(study).show()

In [35]:
# 3. Slice Plot
plot_slice(study).show()

In [36]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [38]:
!pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 3.4 MB/s eta 0:00:00


In [39]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2024-11-14 15:15:45,845] A new study created in memory with name: no-name-af235d22-d273-4c03-bf11-3542efb454bc


[0]	train-mlogloss:0.85409	eval-mlogloss:0.84615
[1]	train-mlogloss:0.68424	eval-mlogloss:0.67042
[2]	train-mlogloss:0.55791	eval-mlogloss:0.53695
[3]	train-mlogloss:0.46305	eval-mlogloss:0.43470
[4]	train-mlogloss:0.38825	eval-mlogloss:0.35502
[5]	train-mlogloss:0.32978	eval-mlogloss:0.29349
[6]	train-mlogloss:0.28348	eval-mlogloss:0.24333
[7]	train-mlogloss:0.24507	eval-mlogloss:0.20464
[8]	train-mlogloss:0.21525	eval-mlogloss:0.17207
[9]	train-mlogloss:0.19197	eval-mlogloss:0.14838
[10]	train-mlogloss:0.17226	eval-mlogloss:0.12999
[11]	train-mlogloss:0.15740	eval-mlogloss:0.11368
[12]	train-mlogloss:0.14529	eval-mlogloss:0.09901
[13]	train-mlogloss:0.13503	eval-mlogloss:0.08881
[14]	train-mlogloss:0.13187	eval-mlogloss:0.08488
[15]	train-mlogloss:0.12924	eval-mlogloss:0.08047
[16]	train-mlogloss:0.12702	eval-mlogloss:0.07832
[17]	train-mlogloss:0.12494	eval-mlogloss:0.07522
[18]	train-mlogloss:0.12310	eval-mlogloss:0.07294
[19]	train-mlogloss:0.12114	eval-mlogloss:0.07008
[20]	train

[I 2024-11-14 15:15:51,753] Trial 0 finished with value: 1.0 and parameters: {'lambda': 0.020269311970445274, 'alpha': 3.2697003153098243e-07, 'eta': 0.19921099838758122, 'gamma': 0.027483243909581775, 'max_depth': 6, 'min_child_weight': 5, 'subsample': 0.9765462615103706, 'colsample_bytree': 0.8963206908772281}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.75929	eval-mlogloss:0.74111
[1]	train-mlogloss:0.56219	eval-mlogloss:0.53931
[2]	train-mlogloss:0.42684	eval-mlogloss:0.39604
[3]	train-mlogloss:0.34563	eval-mlogloss:0.30437
[4]	train-mlogloss:0.28489	eval-mlogloss:0.23849
[5]	train-mlogloss:0.24284	eval-mlogloss:0.19105
[6]	train-mlogloss:0.21580	eval-mlogloss:0.15964
[7]	train-mlogloss:0.20600	eval-mlogloss:0.14624
[8]	train-mlogloss:0.20541	eval-mlogloss:0.14575
[9]	train-mlogloss:0.19873	eval-mlogloss:0.13396
[10]	train-mlogloss:0.19817	eval-mlogloss:0.13140
[11]	train-mlogloss:0.19809	eval-mlogloss:0.13152
[12]	train-mlogloss:0.19191	eval-mlogloss:0.12349
[13]	train-mlogloss:0.19134	eval-mlogloss:0.12335
[14]	train-mlogloss:0.19131	eval-mlogloss:0.12330
[15]	train-mlogloss:0.19070	eval-mlogloss:0.12287
[16]	train-mlogloss:0.19036	eval-mlogloss:0.12315
[17]	train-mlogloss:0.18976	eval-mlogloss:0.12315
[18]	train-mlogloss:0.18938	eval-mlogloss:0.12312
[19]	train-mlogloss:0.18912	eval-mlogloss:0.12306
[20]	train

[I 2024-11-14 15:16:01,514] Trial 1 finished with value: 1.0 and parameters: {'lambda': 0.0001770875741261238, 'alpha': 0.0004117567101723674, 'eta': 0.2907973017947161, 'gamma': 0.1133570063034978, 'max_depth': 6, 'min_child_weight': 9, 'subsample': 0.9198196094474983, 'colsample_bytree': 0.7986585015374492}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:0.87696	eval-mlogloss:0.87724
[1]	train-mlogloss:0.72217	eval-mlogloss:0.71796


[I 2024-11-14 15:16:53,756] Trial 48 pruned. Trial was pruned at iteration 2.


[0]	train-mlogloss:1.05624	eval-mlogloss:1.05736
[1]	train-mlogloss:1.01662	eval-mlogloss:1.01882
[2]	train-mlogloss:0.97488	eval-mlogloss:0.97518
[3]	train-mlogloss:0.94148	eval-mlogloss:0.94011
[4]	train-mlogloss:0.90431	eval-mlogloss:0.90066
[5]	train-mlogloss:0.87479	eval-mlogloss:0.87025
[6]	train-mlogloss:0.84134	eval-mlogloss:0.83459
[7]	train-mlogloss:0.80983	eval-mlogloss:0.80045
[8]	train-mlogloss:0.78479	eval-mlogloss:0.77480


[I 2024-11-14 15:16:53,895] Trial 49 pruned. Trial was pruned at iteration 8.


Best trial: {'lambda': 0.020269311970445274, 'alpha': 3.2697003153098243e-07, 'eta': 0.19921099838758122, 'gamma': 0.027483243909581775, 'max_depth': 6, 'min_child_weight': 5, 'subsample': 0.9765462615103706, 'colsample_bytree': 0.8963206908772281}
Best accuracy: 1.0


In [40]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()